## 1.迭代
如果给定一个list或tuple，我们可以通过for循环来遍历这个list或tuple，这种遍历我们称为迭代（Iteration）。

## 2.迭代器
Python的Iterator对象表示的是一个数据流，Iterator对象可以被next()函数调用并不断返回下一个数据，直到没有数据时抛出StopIteration错误。可以把这个数据流看做是一个有序序列，但我们却不能提前知道序列的长度，只能不断通过next()函数实现按需计算下一个数据，所以Iterator的计算是惰性的，只有在需要返回下一个数据时它才会计算。

## 3.生成器
在Python中，这种一边循环一边计算的机制，称为生成器：generator。

要创建一个generator，有很多种方法。第一种方法很简单，只要把一个列表生成式的[]改成()，就创建了一个generator

如果一个函数定义中包含yield关键字，那么这个函数就不再是一个普通函数，而是一个generator.

函数是顺序执行，遇到return语句或者最后一行函数语句就返回。而变成generator的函数，在每次调用next()的时候执行，遇到yield语句返回，再次执行时从上次返回的yield语句处继续执行。

## 4.map()
map()函数接收两个参数，一个是函数，一个是Iterable，map将传入的函数依次作用到序列的每个元素，并把结果作为新的Iterator返回。

Iterator是惰性序列，因此通过list()函数让它把整个序列都计算出来并返回一个list。

## 5.reduce()
reduce把一个函数作用在一个序列[x1, x2, x3, ...]上，这个函数必须接收两个参数，reduce把结果继续和序列的下一个元素做累积计算

reduce(f, [x1, x2, x3, x4]) = f(f(f(x1, x2), x3), x4)

## 6.filter()
filter()把传入的函数依次作用于每个元素，然后根据返回值是True还是False决定保留还是丢弃该元素。

可见用filter()这个高阶函数，关键在于正确实现一个“筛选”函数。

注意到filter()函数返回的是一个Iterator，也就是一个惰性序列，所以要强迫filter()完成计算结果，需要用list()函数获得所有结果并返回list。

## 7.返回函数

In [1]:
#python引用变量的顺序： 当前作用域局部变量->外层作用域变量->当前模块中的全局变量->python内置变量

##counter

##内层函数能访问外层函数的变量，但不能修改它的指向

def createCounter():
    count = [0]
    def counter():
        count[0] += 1
        return count[0]
    return counter
##这种情况可行是因为count指向的是一个列表的实例对象，实质上，列表的实例对象的地址一直没变，只是其内容的指向改变了而已

###而nonlocal关键字用来在函数或其他作用域中修改外层(非全局)变量

def createCounter():
    count = 0
    def counter():
        nonlocal count 
        count += 1
        return count
    return counter
###global关键字则是用于修改全局变量

In [2]:
def createCounter():
    f = 0
    def counter():

    #nonlocal  f
    f += 1
    return f
return counter
#报错原因，闭包外的变量f在闭包内部只能读取不能修改。

#nonlocal用于声明，修改嵌套作用域（enclosing 作用域，外层非全局作用域）中的变量，  

#所以需要加上nonlocal f

IndentationError: expected an indented block (<ipython-input-2-5c22f59e6280>, line 6)

## 7.闭包

闭包（Closure）是词法闭包（Lexical Closure）的简称，是引用了自由变量的函数。这个被引用的自由变量将和这个函数一同存在，即使已经离开了创造它的环境也不例外。所以，闭包是由函数和与其相关的引用环境组合而成的实体。

在Python中创建一个闭包可以归结为以下三点：

+ 闭包函数必须有内嵌函数
+ 内嵌函数需要引用该嵌套函数上一级namespace中的变量
+ 闭包函数必须返回内嵌函数

通过这三点，就可以创建一个闭包，是不是想到了上一篇中介绍的Python装饰器。没错，Python装饰器就是使用了闭包

In [6]:
def greeting_conf(prefix):
    def greeting(name):
        print (prefix, name)
    return greeting
 
mGreeting = greeting_conf("Good Morning")
mGreeting("Wilber")
mGreeting("Will")

print('\n')
 
aGreeting = greeting_conf("Good Afternoon")    
aGreeting("Wilber")
aGreeting("Will")

Good Morning Wilber
Good Morning Will


Good Afternoon Wilber
Good Afternoon Will


## 8.装饰器

代码运行期间动态增加功能的方式，称之为“装饰器”（Decorator）。

本质上，decorator就是一个返回函数的高阶函数。

In [10]:
def now():
     print('2018-2-8')

In [11]:
f=now

In [12]:
f()

2018-2-8


In [13]:
now.__name__

'now'

In [14]:
def log(func):
    def wrapper(*args,**kw):
        print('call %s()' % func.__name__)
        return func(*args,**kw)
    return wrapper

In [15]:
@log
def now():
     print('2018-2-8')

In [16]:
now()

call now()
2018-2-8


把@log放到now()函数的定义处，相当于执行了语句：

now = log(now)

如果decorator本身需要传入参数，那就需要编写一个返回decorator的高阶函数，写出来会更复杂。比如，要自定义log的文本：

In [17]:
def log(text):
    def decorator(func):
        def wrapper(*args, **kw):
            print('%s %s():' % (text, func.__name__))
            return func(*args, **kw)
        return wrapper
    return decorator

In [18]:
@log('execute')
def now():
    print('2018-2-8')

In [19]:
now()

execute now():
2015-3-25


In [20]:
now.__name__

'wrapper'

因为返回的那个wrapper()函数名字就是'wrapper'，所以，需要把原始函数的__name__等属性复制到wrapper()函数中，否则，有些依赖函数签名的代码执行就会出错。

不需要编写wrapper.__name__ = func.__name__这样的代码，Python内置的functools.wraps就是干这个事的，所以，一个完整的decorator的写法如下：

In [21]:
import functools

def log(func):
    @functools.wraps(func)
    def wrapper(*args, **kw):
        print('call %s():' % func.__name__)
        return func(*args, **kw)
    return wrapper

In [22]:
@log
def now():
     print('2018-2-8')

In [23]:
now()

call now():
2018-2-8


In [24]:
now.__name__

'now'

In [25]:
import functools

def log(text):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kw):
            print('%s %s():' % (text, func.__name__))
            return func(*args, **kw)
        return wrapper
    return decorator

## 9.偏函数

In [26]:
import functools
int2 = functools.partial(int, base=2)

In [27]:
int2('1000000')

64

简单总结functools.partial的作用就是，把一个函数的某些参数给固定住（也就是设置默认值），返回一个新的函数，调用这个新函数会更简单。

注意到上面的新的int2函数，仅仅是把base参数重新设定默认值为2，但也可以在函数调用时传入其他值：

In [28]:
int2('1000000', base=10)

1000000

最后，创建偏函数时，实际上可以接收函数对象、*args和**kw这3个参数

In [29]:
kw = { 'base': 2 }
int('10010', **kw)

18

In [30]:
max2 = functools.partial(max, 10)

实际上会把10作为*args的一部分自动加到左边，也就是：

In [31]:
max2(5, 6, 7)

10